# STA 663 Midterm Exams

Please observe the Duke honor code for this **closed book** exam.

**Permitted exceptions to the closed book rule**

- You may use any of the links accessible from the Help Menu for reference - that is, you may follow a chain of clicks from the landing pages of the sites accessible through the Help Menu. If you find yourself outside the help/reference pages of `python`, `ipython`, `numpy`, `scipy`, `matplotlib`, `sympy`, `pandas`, (e.g. on a Google search page or stackoverflow or current/past versions of the STA 663 notes) you are in danger of violating the honor code and should exit immediately.

- You may also use TAB or SHIFT-TAB completion, as well as `?foo`, `foo?` and `help(foo)` for any function, method or class `foo`.

The total points allocated is 125, but the maximum possible is 100. Hence it is possible to score 100 even with some errors or incomplete solutions.

## Imports

All the necessary packages have been imported for you in the Code cells below. You should not need any additional imports.

In [1]:
%matplotlib inline
import  matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.linalg as la
from collections import Counter
from functools import reduce

In [2]:
%load_ext rpy2.ipython

**1**. (10 points)

Read the flights data at https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv into a `pnadas` data frame. Find the average number of passengers per quarter (Q1, Q2, Q3,Q4) across the years 1950-1959 (inclusive of 1950 and 1959), where

- Q1 = Jan, Feb, Mar
- Q2 = Apr, May, Jun
- Q3 = Jul, Aug, Sep
- Q4 = Oct, Nov, Dec

In [3]:
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv '
data = pd.read_csv(url)
data.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [19]:
data = data[(data['year']>=1950)&(data['year']<=1959)]
data['quarter']=np.where((data['month']=='January')|(data['month']=='February')|(data['month']=='March'),'Q1',
                        np.where((data['month']=='June')|(data['month']=='April')|(data['month']=='May'),'Q2',
                        np.where((data['month']=='September')|(data['month']=='July')|(data['month']=='August'),'Q3','Q4')))
data.groupby(['quarter']).mean()['passengers']

quarter
Q1    245.800000
Q2    278.466667
Q3    329.666667
Q4    250.366667
Name: passengers, dtype: float64

**2**. (10 points)

The Collatz sequence is defined by the following rules for finding the next number

```
if the current number is even, divide by 2
if the current number is odd, multiply by 3 and add 1
if the current number is 1, stop
```

- Find the starting integer that gives the longest Collatz sequence for integers in the range(1, 10000). What is the starting number and length of this Collatz sequence?

In [23]:
def collatz(num):
    length = 1
    while num != 1:
        if num % 2==0:
            num /= 2
        else:
            num = num * 3 + 1
        length += 1
    return length

max_len = 1
num = 1
for i in range(2,10000):    
    length = collatz(i)
    if(length > max_len):
        max_len = length
        num = i


print('The number is',num, 'and the length is',max_len)



The number is 6171 and the length is 262


**3**. (10 points)

Recall that a covariance matrix is a matrix whose entries are

![img](https://wikimedia.org/api/rest_v1/media/math/render/svg/4df2969e65403dd04f2c64137d21ff59b5f54190)

Find the sample covariance matrix of the 4 features of the **iris** data set at http://bit.ly/2ow0oJO using basic `numpy` operations on `ndarrasy`. Do **not** use the `np.cov` or equivalent functions in `pandas` (except for checking). Remember to scale by $1/(n-1)$ for the sample covariance.

In [66]:
url = 'http://bit.ly/2ow0oJO'
iris = pd.read_csv(url)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [67]:
iris = iris.drop('species',axis=1)

In [71]:
iris_array = np.array(iris)
def cov(x,y):
    return np.sum((x-x.mean())*(y-y.mean()))/(len(x)-1)
A = np.zeros((iris_array.shape[1],iris_array.shape[1]))
for i in range(iris_array.shape[1]): 
    for j in range(iris_array.shape[1]):
        A[i,j] = cov(iris_array[:,i],iris_array[:,j])
A

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

**4**. (10 points)

How many numbers in `range(100, 1000)` are divisible by 17 after you square them and add 1? Find this out using only **lambda** functions, **map**, **filter** and **reduce** on `xs`, where `xs = range(100, 10000)`.

In pseudo-code, you want to achieve

```python
xs = range(100, 10000)
count(y for y in (x**2 + 1 for x in xs) if y % 17 == 0)
```

In [50]:
xs = range(100, 10000)
reduce(lambda x,y: x+y//y, filter(lambda x: x%17==0 ,map(lambda x: x**2+1,xs)),0)


1164

**5**. (20 points)

- Given the DNA sequence below, create a $4 \times 4$ transition matrix $A$ where $A[i,j]$ is the probability of the base $j$ appearing immediately after base $i$. Note that a *base* is one of the four letters `a`, `c`, `t` or `g`. The letters below should be treated as a single sequence, broken into separate lines just for formatting purposes. You should check that row probabilities sum to 1. (10 points)
- Find the steady state distribution of the 4 bases from the row stochastic transition matrix - that is the, the values of $x$ for which $x^TA = x^T$. Find the solution by solving a set of linear equations. Hint: you need to add a constraint on the values of $x$. Only partial credit will be given for other methods of finding the steady state distribution. (10 points)

```
gggttgtatgtcacttgagcctgtgcggacgagtgacacttgggacgtgaacagcggcggccgatacgttctctaagatc
ctctcccatgggcctggtctgtatggctttcttgttgtgggggcggagaggcagcgagtgggtgtacattaagcatggcc
accaccatgtggagcgtggcgtggtcgcggagttggcagggtttttgggggtggggagccggttcaggtattccctccgc
gtttctgtcgggtaggggggcttctcgtaagggattgctgcggccgggttctctgggccgtgatgactgcaggtgccatg
gaggcggtttggggggcccccggaagtctagcgggatcgggcttcgtttgtggaggagggggcgagtgcggaggtgttct
```

**Part 1**

In [4]:
s = '''
gggttgtatgtcacttgagcctgtgcggacgagtgacacttgggacgtgaacagcggcggccgatacgttctctaagatc
ctctcccatgggcctggtctgtatggctttcttgttgtgggggcggagaggcagcgagtgggtgtacattaagcatggcc
accaccatgtggagcgtggcgtggtcgcggagttggcagggtttttgggggtggggagccggttcaggtattccctccgc
gtttctgtcgggtaggggggcttctcgtaagggattgctgcggccgggttctctgggccgtgatgactgcaggtgccatg
gaggcggtttggggggcccccggaagtctagcgggatcgggcttcgtttgtggaggagggggcgagtgcggaggtgttct
'''
s = s.replace('\n','')

In [5]:
freq = {}
for ch in zip(s,s[1:]):
    freq[ch] = freq.get(ch,0)+1

In [6]:
d = {0:'a',1:'c',2:'t',3:'g'}
A = np.zeros((4,4))
for i in range(4):
    for j in range(4):
        A[i,j] = freq[(d[i],d[j])]
A = A/A.sum(axis=1).reshape(-1,1)
A

array([[0.09433962, 0.20754717, 0.24528302, 0.45283019],
       [0.16666667, 0.23809524, 0.26190476, 0.33333333],
       [0.10204082, 0.2244898 , 0.2755102 , 0.39795918],
       [0.14634146, 0.18902439, 0.22560976, 0.43902439]])

**Part 2**

In [15]:
I = np.eye(4)
X = np.r_[A.T - I, [np.ones(4)]]
b = np.r_[np.zeros(4), 1].reshape(-1,1)
x, _, _, _ = np.linalg.lstsq(X, b, rcond=None)
x

array([[0.1327228 ],
       [0.21062262],
       [0.24825335],
       [0.40840123]])

**6**. (10 points)

- Find the matrix $A$ that results in rotating the standard vectors in $\mathbb{R}^2$ by 30 degrees counter-clockwise and stretches $e_1$ by a factor of 3 and contracts $e_2$ by a factor of $0.5$. 
- What is the inverse of this matrix? How you find the inverse should reflect your understanding.

The effects of the matrix $A$ and $A^{-1}$ are shown in the figure below:

![image](./vecs.png)

In [22]:
I = np.eye(2)
A = np.array([[1.5*np.sqrt(3),1.5],[-0.25,0.25*np.sqrt(3)]]).T
A

array([[ 2.59807621, -0.25      ],
       [ 1.5       ,  0.4330127 ]])

In [23]:
la.solve(A@I,I)

array([[ 0.28867513,  0.16666667],
       [-1.        ,  1.73205081]])

**7**. (55 points) 

We observe some data points $(x_i, y_i)$, and believe that an appropriate model for the data is that

$$
f(x) = ax^2 + bx^3 + c\sin{x}
$$

with some added noise. Find optimal values of the parameters $\beta = (a, b, c)$ that minimize $\Vert y - f(x) \Vert^2$

1. using `scipy.linalg.lstsq` (10 points)
2. solving the normal equations $X^TX \beta = X^Ty$ (10 points)
3. using `scipy.linalg.svd` (10 points)
4. using gradient descent with RMSProp (no bias correction) and starting with an initial value of $\beta = \begin{bmatrix}1 & 1 & 1\end{bmatrix}$. Use a learning rate of 0.01 and 10,000 iterations, and set the $\beta$ parameter of RMSprop to be 0.9 (this is a different $\beta$ from the parameters of the function we are minimizing). Running gradient descent should take a few seconds to complete. (25 points)

In each case, plot the data and fitted curve using `matplotlib`.

Data
```
x = array([ 3.4027718 ,  4.29209002,  5.88176277,  6.3465969 ,  7.21397852,
        8.26972154, 10.27244608, 10.44703778, 10.79203455, 14.71146298])
y = array([ 25.54026428,  29.4558919 ,  58.50315846,  70.24957254,
        90.55155435, 100.56372833,  91.83189927,  90.41536733,
        90.43103028,  23.0719842 ])
```

In [28]:
x = np.array([ 3.4027718 ,  4.29209002,  5.88176277,  6.3465969 ,  7.21397852,
        8.26972154, 10.27244608, 10.44703778, 10.79203455, 14.71146298])
y = np.array([ 25.54026428,  29.4558919 ,  58.50315846,  70.24957254,
        90.55155435, 100.56372833,  91.83189927,  90.41536733,
        90.43103028,  23.0719842 ])
x_2 = x**2
x_3 = x**3
sinx = np.sin(x)
X = np.c_[x_2,x_3,sinx]
X

array([[ 1.15788559e+01,  3.94002044e+01, -2.58219888e-01],
       [ 1.84220367e+01,  7.90690400e+01, -9.12966995e-01],
       [ 3.45951333e+01,  2.03480367e+02, -3.90728191e-01],
       [ 4.02792922e+01,  2.55636431e+02,  6.33691047e-02],
       [ 5.20414861e+01,  3.75426163e+02,  8.02093898e-01],
       [ 6.83882943e+01,  5.65552151e+02,  9.14817747e-01],
       [ 1.05523148e+02,  1.08398085e+03, -7.49739362e-01],
       [ 1.09140598e+02,  1.14019595e+03, -8.53288558e-01],
       [ 1.16468010e+02,  1.25692678e+03, -9.79357202e-01],
       [ 2.16427143e+02,  3.18395990e+03,  8.39574933e-01]])

### Using `lstsq`

In [31]:
la.lstsq(X,y)[0]

array([ 2.99259014, -0.19883227, 10.20024689])

### Using normal equations

In [32]:
la.solve(X.T@X,X.T@y)

array([ 2.99259014, -0.19883227, 10.20024689])

### Using SVD

In [64]:
U,S,VT = la.svd(X)
VT.T @ np.diag(1/S) @ U.T[:len(S),:] @ y

array([ 2.99259014, -0.19883227, 10.20024689])

### Using Gradient descent with RMSprop

In [ ]:
def grad(X,y,beta):
    return X.T @ X @ beta - X.T @ y

def v(beta,grd,v_k):
    return beta*v_k +(1-beta)@grd**2


def gd(X,y,rate = 0.01, max_iter=10000):
    beta = np.zeros(X.shape[1])
    for i in range(max_iter):
        grd = grad(X,y,beta)
        v = v(beta,grd,v_k)
        beta = beta - rate * grd/np.sqrt(v)
    
    return beta